In [1]:
import pandas as pd
import numpy as np
try:
    import urllib2 as urllib
except ImportError:
    import urllib.request as urllib
import json
import csv

In [2]:
location = pd.read_csv('lat_long_for_API.csv')
location.drop(['Unnamed: 0'], axis=1, inplace=True)

In [3]:
location.tail()

,LocationID_pickup,lon_pickup,lat_pickup,LocationID_dropoff,lon_dropoff,lat_dropoff
68893,263,-73.951209,40.778495,258,-73.856646,40.690140
68894,263,-73.951209,40.778495,259,-73.856399,40.899080
68895,263,-73.951209,40.778495,260,-73.903715,40.746790
68896,263,-73.951209,40.778495,261,-74.012919,40.708976
68897,263,-73.951209,40.778495,262,-73.945830,40.776536


In [29]:
location1 = location.loc[:1000]
location2 = location.loc[1001:30000]
location3 = location.loc[30001:]

In [30]:
#for chech purpose
len(location1), len(location1) + len(location2), len(location1) + len(location2) + len(location3)

(1001, 30001, 68898)

https://transit.api.here.com/v3/route.json
?dep=40.689515%2C-74.176778
&arr=40.625724%2C-73.826141
&time=2019-06-24T07%3A30%3A00
&app_id=CZHAUHoFYaKRZeok8OaM
&app_code=5mYVXKBCV5ROOwoURif4og
&modes=bus%2Csubway
&routing=tt
&max=1
&graph=1

https://transit.api.here.com/v3/route.json?dep=41.9773%2C-87.9019&arr=41.8961%2C-87.6552&time=2019-06-24T07%3A30%3A00&app_id=devportal-demo-20180625&app_code=9v2BkviRwi9Ot26kp2IysQ&routing=tt&forward=6

https://transit.api.here.com/v3/route.json?dep=41.9773%2C-87.9019&arr=41.8961%2C-87.6552&time=2019-06-24T07%3A30%3A00&app_id=devportal-demo-20180625&app_code=9v2BkviRwi9Ot26kp2IysQ&routing=tt&graph=1

In [35]:
lon_pickup = -74.176778
lat_pickup = 40.689515
lon_dropoff = -73.826141
lat_dropoff = 40.625724
appID = 'CZHAUHoFYaKRZeok8OaM'
appCode = '5mYVXKBCV5ROOwoURif4og'
url = str('https://route.api.here.com/routing/7.2/calculateroute.json?waypoint0={}%2C{}&waypoint1={}%2C{}&mode=fastest%3Bpedestrian&app_id={}&app_code={}'.format(lat_pickup, lon_pickup, lat_dropoff, lon_dropoff, appID, appCode))
respond = urllib.urlopen(url)
data = respond.read().decode('utf-8')
data = json.loads(data)
distance = data['response']['route'][0]['summary']['distance']
time = data['response']['route'][0]['summary']['travelTime']

In [6]:
def getWalkinginfo(lat_pickup, lon_pickup, lat_dropoff, lon_dropoff, appID, appCode):
    import pandas as pd
    try:
        import urllib2 as urllib
    except ImportError:
        import urllib.request as urllib
    import json
    
    try:
        url = str('https://route.api.here.com/routing/7.2/calculateroute.json?waypoint0={}%2C{}&waypoint1={}%2C{}&mode=fastest%3Bpedestrian&app_id={}&app_code={}'.format(lat_pickup, lon_pickup, lat_dropoff, lon_dropoff, appID, appCode))
        respond = urllib.urlopen(url)
        data = respond.read().decode('utf-8')
        data = json.loads(data)
          
        timespend = data['response']['route'][0]['summary']['travelTime']
        distance =  data['response']['route'][0]['summary']['distance']
    except:
        data = 'N/A'
        timespend = 'N/A'
        distance = 'N/A'
    
    return data, timespend, distance

In [7]:
import time, sys
from IPython.display import clear_output
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [8]:
def walking(file, data_list, time_list, distance_list, appID, appCode):
    for index, row in file.iterrows():
        lon_pickup = row[1]
        lat_pickup = row[2]
        lon_dropoff = row[4]
        lat_dropoff = row[5]

        data, timespend, distance = getWalkinginfo(lat_pickup, lon_pickup, lat_dropoff, lon_dropoff, appID, appCode)

        data_list.append(data)
        time_list.append(timespend)
        distance_list.append(distance)
       
        time.sleep(0.0001) 
        update_progress(index / len(file))
    update_progress(1)
    return data_list, time_list, distance_list

### 1st CHUNK:

In [52]:
data_list = []
time_list = []
distance_list = []
appID = 'CZHAUHoFYaKRZeok8OaM'
app_code = '5mYVXKBCV5ROOwoURif4og'
data_list1, time_list1, distance_list1 = walking(location1, data_list, time_list, distance_list, appID, app_code)

Progress: [####################] 100.0%


In [58]:
print(len(time_list1))
type(data_list1[0])

1001


dict

In [54]:
w1 = pd.DataFrame(np.column_stack([time_list1, distance_list1]), 
                   columns=['time_list1', 'distance_list1'])
w1.to_csv('walking1.csv')

In [67]:
data = {}
data['data1'] = data_list1
data1 = pd.DataFrame(data)
data1.to_csv('walkingoriginal1.csv')


### 2nd CHUNK

In [68]:
data_list = []
time_list = []
distance_list = []
appID = 'CZHAUHoFYaKRZeok8OaM'
app_code = '5mYVXKBCV5ROOwoURif4og'
data_list2, time_list2, distance_list2 = walking(location2, data_list, time_list, distance_list, appID, app_code)

Progress: [####################] 100.0%


In [70]:
print(len(time_list2))
time_list2[:10]

29000


[5238, 22366, 22178, 17210, 10034, 14942, 7415, 1105, 8245, 7386]

In [71]:
w2 = pd.DataFrame(np.column_stack([time_list2, distance_list2]), 
                   columns=['time_list2', 'distance_list2'], index=location2.index)
w2.to_csv('walking2.csv')

In [72]:
data = {}
data['data2'] = data_list2
data2 = pd.DataFrame(data)
data2.to_csv('walkingoriginal2.csv')


### 3rd CHUNK

In [9]:
data_list = []
time_list = []
distance_list = []
appID = 'CZHAUHoFYaKRZeok8OaM'
app_code = '5mYVXKBCV5ROOwoURif4og'
data_list3, time_list3, distance_list3 = walking(location3, data_list, time_list, distance_list, appID, app_code)

Progress: [####################] 100.0%


In [28]:
print(len(time_list3))
time_list3[:10]

38897


[11955, 9394, 10967, 12738, 16770, 9685, 19004, 21798, 14047, 16109]

In [43]:
w3 = pd.DataFrame(np.column_stack([time_list3, distance_list3]), 
                   columns=['time_list3', 'distance_list3'], index=location3.index)
w3.to_csv('walking3.csv')

,time_list3,distance_list3
30001,11955,23150
30002,9394,14898
30003,10967,22007
30004,12738,23770
30005,16770,39153


In [52]:
w3.head()

,delete,time,distance
0,30001,11955.0,23150.0
1,30002,9394.0,14898.0
2,30003,10967.0,22007.0
3,30004,12738.0,23770.0
4,30005,16770.0,39153.0


In [15]:
data = {}
data['data3'] = data_list3
data3 = pd.DataFrame(data)
data3.to_csv('walkingoriginal3.csv')


In [53]:
w1 = pd.read_csv('walking1.csv').rename(columns={'Unnamed: 0':'index', 'distance_list1': 'distance', 'time_list1' : 'time'})[['time', 'distance']]
w2 = pd.read_csv('walking2.csv').rename(columns={'Unnamed: 0':'index', 'distance_list2': 'distance', 'time_list2' : 'time'})
w3 = pd.read_csv('walking3.csv').rename(columns={'Unnamed: 0':'index', 'distance_list3': 'distance', 'time_list3' : 'time'}).set_index('index')

In [50]:
w123 = [w1, w2, w3]
wdf123 = pd.concat(w123)[['time', 'distance']]
print(wdf123.shape)
wdf123.head()

(68898, 2)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,time,distance
0,31379.0,65155.0
1,34145.0,71302.0
2,25529.0,49169.0
3,42701.0,50553.0
4,31821.0,39772.0


### Aggregated data

In [23]:
location1.head()

,LocationID_pickup,lon_pickup,lat_pickup,LocationID_dropoff,lon_dropoff,lat_dropoff
0,1,-74.176778,40.689515,2,-73.826141,40.625724
1,1,-74.176778,40.689515,3,-73.849479,40.865871
2,1,-74.176778,40.689515,4,-73.977024,40.724151
3,1,-74.176778,40.689515,5,-74.189938,40.550339
4,1,-74.176778,40.689515,6,-74.067786,40.599053


In [24]:
w1.head()

,time,distance
0,31379.0,65155.0
1,34145.0,71302.0
2,25529.0,49169.0
3,42701.0,50553.0
4,31821.0,39772.0


In [25]:
combine1 = pd.merge(location1, w1, left_index =True, right_index=True)
#combine1['LocationID_pickup'] = combine1['LocationID_pickup'].astype(float)
print(combine1.shape)
combine1 = combine1.dropna()
combine1['LocationID_pickup'] = combine1['LocationID_pickup'].astype(int)
print(combine1.shape)
combine1.head()

(1001, 8)
(997, 8)


,LocationID_pickup,lon_pickup,lat_pickup,LocationID_dropoff,lon_dropoff,lat_dropoff,time,distance
0,1,-74.176778,40.689515,2,-73.826141,40.625724,31379.0,65155.0
1,1,-74.176778,40.689515,3,-73.849479,40.865871,34145.0,71302.0
2,1,-74.176778,40.689515,4,-73.977024,40.724151,25529.0,49169.0
3,1,-74.176778,40.689515,5,-74.189938,40.550339,42701.0,50553.0
4,1,-74.176778,40.689515,6,-74.067786,40.599053,31821.0,39772.0


In [56]:
combine2 = pd.merge(location2, w2, left_index =True, right_index=True)
#combine1['LocationID_pickup'] = combine1['LocationID_pickup'].astype(float)
print(combine2.shape)
combine2 = combine2.dropna()
combine2['LocationID_pickup'] = combine2['LocationID_pickup'].astype(int)
print(combine2.shape)
combine2.head()
combine2.head()

(27999, 9)
(27892, 9)


,LocationID_pickup,lon_pickup,lat_pickup,LocationID_dropoff,lon_dropoff,lat_dropoff,index,time,distance
1001,4,-73.977024,40.724151,217,-73.956841,40.703251,1001,4691.0,7031.0
1002,4,-73.977024,40.724151,218,-73.772117,40.673302,1002,11211.0,11136.0
1003,4,-73.977024,40.724151,219,-73.761009,40.660257,1003,17586.0,31596.0
1004,4,-73.977024,40.724151,220,-73.911881,40.881892,1004,6855.0,6776.0
1005,4,-73.977024,40.724151,221,-74.081329,40.620232,1005,14818.0,29653.0


In [54]:
w3.head()

,time,distance
index,,
30001,11955.0,23150.0
30002,9394.0,14898.0
30003,10967.0,22007.0
30004,12738.0,23770.0
30005,16770.0,39153.0


In [55]:
combine3 = pd.merge(location3, w3, left_index =True, right_index=True)
print(combine3.shape)
combine3 = combine3.dropna()
combine3['LocationID_pickup'] = combine3['LocationID_pickup'].astype(int)
print(combine3.shape)
combine3.head()

(38897, 8)
(38488, 8)


,LocationID_pickup,lon_pickup,lat_pickup,LocationID_dropoff,lon_dropoff,lat_dropoff,time,distance
30001,115,-74.092474,40.620141,143,-73.987973,40.775771,11955.0,23150.0
30002,115,-74.092474,40.620141,144,-73.997407,40.720581,9394.0,14898.0
30003,115,-74.092474,40.620141,145,-73.948693,40.746594,10967.0,22007.0
30004,115,-74.092474,40.620141,146,-73.933639,40.754524,12738.0,23770.0
30005,115,-74.092474,40.620141,147,-73.898192,40.819199,16770.0,39153.0


In [57]:
walkingAll = [combine1, combine2, combine3]
final_walking = pd.concat(walkingAll)[['LocationID_pickup', 'LocationID_dropoff', 'time', 'distance']]
print(final_walking.shape)
final_walking.head()

(67377, 4)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,LocationID_pickup,LocationID_dropoff,time,distance
0,1,2,31379.0,65155.0
1,1,3,34145.0,71302.0
2,1,4,25529.0,49169.0
3,1,5,42701.0,50553.0
4,1,6,31821.0,39772.0


In [58]:
final_walking.to_csv('final_walking.csv')